<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/genpass/genpass_from_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# github代码复制到google colab

参考资料：
1. https://techsupportallbugs.wordpress.com/2018/06/05/using-git-with-colab-via-ssh/

## 配置SSH
1. **上传压缩后的密钥对文件和配置文件**
1. 将密钥对文件和配置文件解压到/root/.ssh文件夹中
2. 将私钥加载到本地ssh-agent中
3. 设置git账户

In [ ]:
!rm -rf ~/.ssh
!unzip '/content/ssh-colab.zip' -d ~/.ssh
!chmod 700 ~/.ssh

# add the ssh server as a hnown host
!touch ~/.ssh/known_hosts
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!chmod 644 ~/.ssh/known_hosts

In [ ]:
!ssh-agent /bin/bash 

# 以下命令在ssh-agent启动的shell中执行
# chmod 600 ~/.ssh/id_rsa_colab  # 私钥需要设置仅自己可以访问，才能添加到代理
# ssh-add ~/.ssh/id_rsa_colab
# ssh-add -l

In [4]:
!git config --global user.name 'colab'
!git config --global user.email 'vyapings@163.com'

## clone私有库
1. 安装google云盘
2. 将工作目录更改为云端硬盘内的文件夹
3. 运行git clone。

In [ ]:
from google.colab import drive
drive.mount('./drive')

import os
os.chdir('/content/drive/MyDrive')

!pwd
!ls

In [ ]:
# !rm -rf gen_pass
# !git clone git@github.com:YapingWu/gen_pass.git

import os
os.chdir('/content/drive/MyDrive/gen_pass')
# !git stash
# !git stash clear
!git pull origin master

# 运行genpass项目

In [ ]:
# 修改时区
!date -R
import os
os.environ['TZ'] = "Asia/Shanghai"
!date -R

In [ ]:
# 修改工作目录
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/gen_pass')
print(os.getcwd())

## 数据分析

In [7]:
!python main.py \
  --stats-only \
  --infile dataset/phpbb.txt dataset/myspace.txt

2021-03-25 04:40:35.212460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-25 04:40:38 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-25 04:40:39 [INFO] Start...
2021-03-25 04:40:39 [INFO] dataset/phpbb.txt数据集中的密码数量：184379
2021-03-25 04:40:39 [INFO] 密码长度不在[4, 20]区间的密码个数：1013，占总密码数量的百分比：0.55%
2021-03-25 04:40:39 [INFO] dataset/myspace.txt数据集中的密码数量：37144
2021-03-25 04:40:39 [INFO] 密码长度不在[4, 20]区间的密码个数：259，占总密码数量的百分比：0.70%
2021-03-25 04:40:39 [INFO] 分析结果图保存到文件：result/figure/password.png
2021-03-25 04:40:40 [INFO] End...


## 数据预处理

In [5]:
!python main.py \
  --pre-processing-only \
  --infile dataset/phpbb.txt dataset/myspace.txt

2021-03-25 13:16:17.712334: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-25 13:16:19 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-25 13:16:19 [INFO] Start...
2021-03-25 13:16:19 [INFO] 原始密码集划分为测试集和训练集
2021-03-25 13:16:20 [INFO] dataset/phpbb.txt：train shape：(147503, 1), test shape：(36876, 1)
2021-03-25 13:16:21 [INFO] dataset/myspace.txt：train shape：(29715, 1), test shape：(7429, 1)
2021-03-25 13:16:21 [INFO] 开始预处理
2021-03-25 13:16:21 [INFO] 待处理的文件名称：dataset/phpbb.txt
2021-03-25 13:16:21 [INFO] PCFG编码
2021-03-25 13:16:22 [INFO] 去重后的编码序列个数：2873， 保存到文件：r

## 训练word-level LSTM

### 训练分词器

In [ ]:
!python main.py \
    --train --model tokenizer \
    --dnames myspace phpbb myspace_part phpbb_part

### 使用pcfg编码结果训练LSTM模型

In [ ]:
!python main.py \
    -e pro \
    --train \
    --model lstm \
    --dnames myspace phpbb myspace_part phpbb_part

## 训练char-level CNN

In [10]:
!python main.py \
    -e pro \
    --train \
    --model cnn \
    --dnames myspace phpbb

2021-03-27 11:14:32.413131: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 11:14:34 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-27 11:14:34 [INFO] Start...
2021-03-27 11:14:34 [INFO] 开始加载数据...
2021-03-27 11:14:34 [INFO] dataset/myspace.txt 数据大小：37144
2021-03-27 11:14:34 [INFO] dataset/phpbb.txt 数据大小：184379
2021-03-27 11:14:34 [INFO] 训练模型的总数据量：77144
2021-03-27 11:14:34 [INFO] 划分验证集和测试集
2021-03-27 11:14:34 [INFO] x_val Shape: (15428, 32), y_val Shape: (15428, 2)
2021-03-27 11:14:34 [INFO] x_test Shape: (15428, 32), y_test Shape: (15428, 2)
2021-03-27 

In [8]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'logs/cnn/cnn210327110825'

## 生成密码

In [ ]:
# 使用genpass模型生成密码
!python main.py \
  -e pro \
  --generate genpass \
  -c 0.3 \
  --alphas 5 6 \
  --chunk 100000

2021-03-27 11:58:42.517885: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 11:58:44 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-27 11:58:44 [INFO] Start...
2021-03-27 11:58:44 [INFO] 创建GenPass模型。用于训练GenPass模型的数据集：['myspace', 'phpbb']，C=0.3
2021-03-27 11:58:44 [INFO] 创建PL模型。数据集名称：myspace
2021-03-27 11:58:44.532119: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 11:58:44.533155: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library lib

In [ ]:
!python main.py \
  -e pro \
  --generate pl \
  --dnames myspace phpbb myspace_part phpbb_part\
  --alphas 6 \
  --chunk 100000

## 评估模型

In [20]:
!python main.py -e pro --evaluate --alphas 3 4 5 6 -c 0.3

2021-03-27 11:56:59.710571: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-27 11:57:01 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-27 11:57:01 [INFO] Start...
2021-03-27 11:57:01 [INFO] 开始评估genpass模型
2021-03-27 11:57:01 [INFO] 创建GenPass模型。用于训练GenPass模型的数据集：['myspace', 'phpbb']，C=0.3
2021-03-27 11:57:01 [INFO] 创建PL模型。数据集名称：myspace
2021-03-27 11:57:01.777093: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-27 11:57:01.778169: I tensorflow/stream_executor/platform/default/dso_loader.cc:49